In [2]:
from htm.algorithms import TemporalMemory
from htm.bindings.sdr import SDR
import numpy as np

In [5]:
def make_sdr(pos: tuple, _shape: tuple) -> SDR:
    result = SDR(_shape)
    numpy_res = result.dense
    numpy_res = np.zeros(_shape)
    numpy_res[pos] = 1
    result.dense = numpy_res
    return result

def make_sdrs(array: np.ndarray, _shape: tuple) -> np.ndarray:
    result = np.ndarray((array.size,), dtype=SDR)
    iterator = 0
    for number in array:
        result[iterator] = make_sdr(number, _shape)
        iterator += 1
    return result

def learn_model(tm: TemporalMemory, sdrs: np.ndarray, num_epochs=10) -> list:
    errors = []
    for epoch in range(num_epochs):
        for sdr in sdrs:
            tm.compute(sdr, learn=True)
            tm.activateDendrites(True)
            errors.append(tm.anomaly)
    return errors

In [33]:

class UnionPooler:

    initial_activation = 2

    def __init__(self, _shape):
        self._unionSDR = SDR(_shape)
        self._pooling_activations = np.zeros(_shape, dtype=float)


    def add_sdr(self, sdr: SDR):

        self._unionSDR.dense[sdr.dense != 0] += self.initial_activation

    def get_state(self):
        return self._unionSDR.sparse

shape = (4, 4)
union_pooler = UnionPooler(shape)
test_sdr = make_sdr((0,1), shape)
union_pooler.add_sdr(test_sdr)
union_pooler.get_state()

array([], dtype=uint32)